In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers;
!pip install sentencepiece;
!pip install Ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00


In [9]:
from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)
import pickle
import json
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.tokenization_kobert import KoBertTokenizer
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [5]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [10]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/standard_to_dialect/no_commit/added_tokens_trg.json", 'rb') as f:
    dw_dict = json.load(f)
dict_swap = {v:k for k,v in dw_dict.items()}
ma=54000
mi=51200
for i in range(51200,54001):
    trg_tokenizer.add_tokens(dict_swap[i])

In [11]:
model = EncoderDecoderModel.from_pretrained('leadawon/ossp-reverse-v0_3')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

In [13]:
from IPython.display import display
import ipywidgets as widgets

dialect = widgets.Textarea(
    placeholder='번역할 표준어',
    description="입력",
    disabled=False
)

button = widgets.Button(
    description='번역!',
    disabled=False,
    tooltip='표준어를 번역합니다.'
)

standard = widgets.Textarea(
    description="출력",
    disabled=True
)

def translate(_):
    standard.value = ""
    text = dialect.get_interact_value()
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    output = model.generate(**embeddings)[0, 1:-1]
    standard.value = trg_tokenizer.decode(output.cpu())

button.on_click(translate)
display(dialect, button, standard)

Textarea(value='', description='입력', placeholder='번역할 표준어')

Button(description='번역!', style=ButtonStyle(), tooltip='표준어를 번역합니다.')

Textarea(value='', description='출력', disabled=True)